# GRU

*Gated Recurent Unit*

ゲート付き回帰型ユニット, ゲート付きRNN

RNN層にゲートと呼ばれる機構を追加して長期的な文脈が保持できるようになったもの。  
通常のRNNは長期的な文脈の保持が苦手とされている。

In [1]:
import os; os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import random
from typing import List

import tensorflow as tf
import tensorflow_datasets as tfds
import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext import transforms
from torchtext.vocab import build_vocab_from_iterator
from torchvision.transforms import Compose
from dlprog import train_progress


---

## ゲート

あるデータをどれくらい通すかを決める機構。0-1の値を出力する。  
NNで実装してみる。

In [2]:
class Gate(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, input_size),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

入力されたデータを線形変化し、sigmoid関数に入力するだけ。  
これを元のデータに掛けることで、元のデータの一部を**通した**ということになる。

In [12]:
input_size = 3
gate = Gate(input_size)

x = torch.randn(input_size)
y = x * gate(x)
print('input:', x)
print('gate:', gate(x))
print('output:', y)

input: tensor([-0.8122, -0.5258,  0.9682])
gate: tensor([0.5705, 0.3989, 0.5374], grad_fn=<SigmoidBackward0>)
output: tensor([-0.4634, -0.2097,  0.5203], grad_fn=<MulBackward0>)



---

## GRU

GRUの構造とその利点を見ていこう。

In [ ]:
class FullyConnected(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_hidden = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        return self.fc_input(x) + self.fc_hidden(h)

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = FullyConnected(input_size, output_size)
        self.gate_update = nn.Sequential(
            FullyConnected(input_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        x = F.tanh(self.fc_input(x, h))
        z = self.gate_update(x)
        h = (1 - z) * h + z * x
        return h

In [ ]:
class GRU(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc_input = FullyConnected(input_size, output_size)
        self.gate_update = nn.Sequential(
            FullyConnected(input_size, output_size),
            nn.Sigmoid()
        )
        self.gate_reset = nn.Sequential(
            FullyConnected(input_size, output_size),
            nn.Sigmoid()
        )

    def forward(self, x, h):
        r = self.gate_reset(x, h)
        x = F.tanh(self.fc_input(x, h * r))
        z = self.gate_update(x)
        h = (1 - z) * h + z * x
        return h